# GPT-2

Use the HuggingFace API to fine-tune the smallest variant of GPT-2 the preprocessed Recipe Box dataset.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -q datasets
!pip install -q transformers

In [3]:
!nvidia-smi

Fri Dec 18 17:42:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import os
import pathlib
import pandas as pd

In [5]:
CACHE_DIR = "./drive/Shared drives/CS 269: Recipe/tmp"
pathlib.Path(CACHE_DIR).mkdir(exist_ok=True)
dataset_path = os.path.join(CACHE_DIR, 'recipes.pkl')

In [6]:
if not os.path.exists(dataset_path):
    raise SystemExit("Run preprocess_pickle.ipynb to generate data file before continuing")
else:
    recipes = pd.read_pickle(dataset_path)

# Subset dataset for feasible training time
recipes = recipes[:20000]

In [7]:
recipes

,title,ingredients,instructions
0,Slow Cooker Chicken and Dumplings,"• 4 skinless, boneless chicken breast halves\n...","‣ Place the chicken, butter, soup, and onion i..."
1,Awesome Slow Cooker Pot Roast,• 2 (10.75 ounce) cans condensed cream of mush...,"‣ In a slow cooker, mix cream of mushroom soup..."
2,Brown Sugar Meatloaf,• 1/2 cup packed brown sugar\n• 1/2 cup ketchu...,‣ Preheat oven to 350 degrees F (175 degrees C...
3,Best Chocolate Chip Cookies,"• 1 cup butter, softened\n• 1 cup white sugar\...",‣ Preheat oven to 350 degrees F (175 degrees C...
4,Homemade Mac and Cheese Casserole,• 8 ounces whole wheat rotini pasta\n• 3 cups ...,‣ Preheat oven to 350 degrees F. Line a 2-quar...
...,...,...,...
20161,Georgia's Tennessee Jam Cake,"• 1 cup butter, softened\n• 2 cups white sugar...",‣ Preheat the oven to 350 degrees F (175 degre...
20162,Poached Eggs and Asparagus,• 4 eggs\n• 1 cube chicken bouillon (optional)...,‣ Fill a saucepan half way full of water. Brin...
20163,Bistecca alla Fiorentina (Tuscan Porterhouse),"• 4 sprigs fresh rosemary, chopped\n• 1 (2 1/2...",‣ Press chopped rosemary onto both sides of po...
20164,Courtney's Three Tomato Pasta Sauce,• 1/2 pound bulk mild Italian sausage\n• 1/2 p...,‣ Cook mild and hot Italian sausage in a large...


# Fine-tuning function

https://github.com/huggingface/transformers/tree/master/examples/language-modeling

In [8]:
# Download fine-tuning script
!curl -O https://raw.githubusercontent.com/huggingface/transformers/master/examples/language-modeling/run_clm.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15593  100 15593    0     0   134k      0 --:--:-- --:--:-- --:--:--  134k


In [9]:
# Show all fine-tuning options
!python run_clm.py --help

2020-12-18 17:42:45.079170: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
usage: run_clm.py [-h] [--model_name_or_path MODEL_NAME_OR_PATH]
                  [--model_type MODEL_TYPE] [--config_name CONFIG_NAME]
                  [--tokenizer_name TOKENIZER_NAME] [--cache_dir CACHE_DIR]
                  [--no_use_fast_tokenizer] [--dataset_name DATASET_NAME]
                  [--dataset_config_name DATASET_CONFIG_NAME]
                  [--train_file TRAIN_FILE]
                  [--validation_file VALIDATION_FILE]
                  [--block_size BLOCK_SIZE] [--overwrite_cache]
                  [--validation_split_percentage VALIDATION_SPLIT_PERCENTAGE]
                  [--preprocessing_num_workers PREPROCESSING_NUM_WORKERS]
                  --output_dir OUTPUT_DIR [--overwrite_output_dir]
                  [--do_train] [--do_eval] [--do_predict] [--model_parallel]
                  [--evaluation_strategy {Evalu

In [10]:
# Hyperparameters for fine-tuning
FINETUNE_PARAMS = [
    "per_device_train_batch_size",  # Batch size per GPU/TPU core/CPU for training.
    "per_device_eval_batch_size",   # Batch size per GPU/TPU core/CPU for evaluation.
    "learning_rate",                # The initial learning rate for Adam.
    "weight_decay",                 # Weight decay if we apply some.
    "adam_epsilon",                 # Epsilon for Adam optimizer.
    "num_train_epochs",             # Total number of training epochs to perform.
    "max_steps",                    # If > 0: set total number of training steps to perform. Override num_train_epochs.
    "warmup_steps",                 # Linear warmup over warmup_steps.
    "save_steps"                    # Save checkpoint every X updates steps.
]

In [11]:
def get_model_path(model_name, nickname):
    model_filename = f"{os.path.basename(model_name)}_{nickname}"
    print(model_filename)
    return os.path.join(CACHE_DIR, model_filename)

Supports fine-tuning of any of the [hugging face causal language models](https://huggingface.co/models?filter=causal-lm) for `model_name`.

In [12]:
from sklearn.model_selection import train_test_split

def finetune(model_name, nickname, dataset, test_size=.2, **kwargs):
    # Split dataset into training and validation sets
    train_recipes, eval_recipes = train_test_split(dataset, test_size=test_size)
    print(f"{len(train_recipes)} training / {len(eval_recipes)} validation")

    # Save datasets to text files for training script
    train_file = f"train_{nickname}.txt"
    validation_file = f"eval_{nickname}.txt"

    with open(train_file, 'w') as f:
        f.write("\n".join(train_recipes))

    with open(validation_file, 'w') as f:
        f.write("\n".join(eval_recipes))

    # Get path to save model to  
    output_dir = get_model_path(model_name, nickname)

    # Parse fine-tuning parameters
    params = [f"--{arg}={value}" for (arg, value) in kwargs.items() if arg in FINETUNE_PARAMS]
    param_str = ' '.join(params)
    print(params)

    # Print out any unrecognized parameters
    unrecognized_params = [arg for arg in kwargs if arg not in FINETUNE_PARAMS]
    if len(unrecognized_params) != 0:
        print(f"Ignoring unrecognized parameters: {unrecognized_params}")
        
    # Fine-tune model
    # NOTE: Add --overwrite_output_dir to replace a model with same name
    # fp16 reduces the memory requirement of your model on the GPU by using half-precision training, basically allowing to double the batch size.
    !python run_clm.py \
        --model_name_or_path="{model_name}" \
        --train_file="{train_file}" \
        --validation_file="{validation_file}" \
        --do_train \
        --do_eval \
        --fp16 \
        --fp16_opt_level=O1 \
        --output_dir="{output_dir}" \
        {param_str}

# Text generation function

In [13]:
# Download generation script
!curl -O https://raw.githubusercontent.com/huggingface/transformers/master/examples/text-generation/run_generation.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11046  100 11046    0     0  77244      0 --:--:-- --:--:-- --:--:-- 76708


In [14]:
# Show all generation options
!python run_generation.py --help

2020-12-18 17:42:48.984353: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
usage: run_generation.py [-h] --model_type MODEL_TYPE --model_name_or_path
                         MODEL_NAME_OR_PATH [--prompt PROMPT]
                         [--length LENGTH] [--stop_token STOP_TOKEN]
                         [--temperature TEMPERATURE]
                         [--repetition_penalty REPETITION_PENALTY] [--k K]
                         [--p P] [--prefix PREFIX]
                         [--padding_text PADDING_TEXT]
                         [--xlm_language XLM_LANGUAGE] [--seed SEED]
                         [--no_cuda]
                         [--num_return_sequences NUM_RETURN_SEQUENCES]
                         [--fp16]

optional arguments:
  -h, --help            show this help message and exit
  --model_type MODEL_TYPE
                        Model type selected in the list: gpt2, ctrl, openai-
                       

In [15]:
# Parameters for text generation
GENERATE_PARAMS = [
    "temperature",  # Temperature of 1.0 has no effect, lower tends toward greedy sampling
    "length",       # Length of generated text
    "prefix",       # Text added prior to input
    "stop_token"    # Token at which text generation is stopped
]

In [16]:
def generate(model_name, model_path, prompt, **kwargs):
    # Parse text generation parameters
    params = [f"--{arg}='{value}'" for (arg, value) in kwargs.items() if arg in GENERATE_PARAMS]
    param_str = ' '.join(params)
    print(params)

    # Generate text based on prompt
    !python run_generation.py \
        --model_type="{model_name}" \
        --model_name_or_path="{model_path}" \
        --prompt="{prompt}" \
        {param_str}


# Generate recipe from title

In [17]:
def recipe_to_str(recipe):
    # Combine components of recipe into a string
    return f"<BOS>{recipe.title}<ING>{recipe.ingredients}<INS>{recipe.instructions}<EOS>"

recipe_strings = recipes.apply(recipe_to_str, axis=1)

In [18]:
recipe_strings

0        <BOS>Slow Cooker Chicken and Dumplings<ING>• 4...
1        <BOS>Awesome Slow Cooker Pot Roast<ING>• 2 (10...
2        <BOS>Brown Sugar Meatloaf<ING>• 1/2 cup packed...
3        <BOS>Best Chocolate Chip Cookies<ING>• 1 cup b...
4        <BOS>Homemade Mac and Cheese Casserole<ING>• 8...
                               ...                        
20161    <BOS>Georgia's Tennessee Jam Cake<ING>• 1 cup ...
20162    <BOS>Poached Eggs and Asparagus<ING>• 4 eggs\n...
20163    <BOS>Bistecca alla Fiorentina (Tuscan Porterho...
20164    <BOS>Courtney's Three Tomato Pasta Sauce<ING>•...
20165    <BOS>Tom Collins<ING>• 2 fluid ounces gin\n• 2...
Length: 20000, dtype: object

## GPT-2

In [19]:
finetune('gpt2', 'fp16_recipe_20000_lr_5e-5_wd_1e-1', recipe_strings,
         per_device_train_batch_size=2, # maximum supported in GPU memory
         per_device_eval_batch_size=2,  # maximum supported in GPU memory
         learning_rate=5e-5,
         weight_decay=1e-1,
         num_train_epochs=10,
         save_steps=5000)

16000 training / 4000 validation
gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1
['--per_device_train_batch_size=2', '--per_device_eval_batch_size=2', '--learning_rate=5e-05', '--weight_decay=0.1', '--num_train_epochs=10', '--save_steps=5000']
2020-12-18 17:42:53.242622: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Traceback (most recent call last):
  File "run_clm.py", line 374, in <module>
    main()
  File "run_clm.py", line 159, in main
    f"Output directory ({training_args.output_dir}) already exists and is not empty."
ValueError: Output directory (./drive/Shared drives/CS 269: Recipe/tmp/gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1) already exists and is not empty.Use --overwrite_output_dir to overcome.


## Generate recipes

In [20]:
def generate_recipe(title, ingredients=[], generate_more_ingredients=True):
    ingredients = '\n'.join(map(lambda i: f"• {i}", ingredients))
    prompt = f"{title}<ING>{ingredients}"

    if not generate_more_ingredients:
        prompt += "\n<INS>"

    model_path = get_model_path('gpt2', 'fp16_recipe_20000_lr_5e-5_wd_1e-1')
    generate('gpt2', model_path, prompt, temperature=0.7, length=400, prefix='<BOS>', stop_token='<EOS>')

In [21]:
generate_recipe("Taco")

gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1
["--temperature='0.7'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2020-12-18 17:42:56.648191: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
12/18/2020 17:42:58 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
12/18/2020 17:43:08 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/CS 269: Recipe/tmp/gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Taco<ING>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.7, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Taco<ING>• 1 (16 ounce) can refried beans• 2 (14.5 ounce) cans Mexican-style corn• 1 (10 ounce) can refried be

In [22]:
generate_recipe("Casserole", ['pasta', 'tuna', 'cheese', 'jalapeno', 'ketchup'])

gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1
["--temperature='0.7'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2020-12-18 17:43:16.322632: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
12/18/2020 17:43:18 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
12/18/2020 17:43:28 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/CS 269: Recipe/tmp/gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Casserole<ING>• pasta\n• tuna\n• cheese\n• jalapeno\n• ketchup', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.7, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Casserole<ING>• pasta
• tuna
• cheese
• jalapeno
• ketchu

In [23]:
generate_recipe("Casserole", ['pasta', 'tuna', 'cheese', 'jalapeno', 'ketchup'], False)

gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1
["--temperature='0.7'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2020-12-18 17:43:35.878057: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
12/18/2020 17:43:37 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
12/18/2020 17:43:47 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/CS 269: Recipe/tmp/gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Casserole<ING>• pasta\n• tuna\n• cheese\n• jalapeno\n• ketchup\n<INS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.7, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Casserole<ING>• pasta
• tuna
• cheese
• jalapeno
•

In [24]:
generate_recipe("Texas Pecan and Chocolate Pie", ['chocolate'])

gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1
["--temperature='0.7'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2020-12-18 17:43:55.313477: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
12/18/2020 17:43:57 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
12/18/2020 17:44:07 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/CS 269: Recipe/tmp/gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Texas Pecan and Chocolate Pie<ING>• chocolate', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.7, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Texas Pecan and Chocolate Pie<ING>• chocolate-covered pretzels• 1 (9 inch)

In [25]:
generate_recipe("Mushroom Tacos", ['tortilla', 'mushroom', 'cheese', 'lettuce'], False)

gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1
["--temperature='0.7'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2020-12-18 17:44:14.827157: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
12/18/2020 17:44:16 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
12/18/2020 17:44:26 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/CS 269: Recipe/tmp/gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Mushroom Tacos<ING>• tortilla\n• mushroom\n• cheese\n• lettuce\n<INS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.7, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Mushroom Tacos<ING>• tortilla
• mushroom
• cheese


# Generate evaluation recipes

In [26]:
# Set to True for generating recipes for evaluation
GENERATE_EVAL_RECIPES = True 

In [27]:
def generate_file(model_name, model_path, prompt, output_path, **kwargs):
    # Parse text generation parameters
    params = [f"--{arg}='{value}'" for (arg, value) in kwargs.items() if arg in GENERATE_PARAMS]
    param_str = ' '.join(params)
    print(params)

    # Generate text based on prompt
    !python run_generation.py \
        --model_type="{model_name}" \
        --model_name_or_path="{model_path}" \
        --prompt="{prompt}" \
        {param_str} \
        | tail -n +2 > "{output_path}"


In [28]:
import random
from tqdm import tqdm

if GENERATE_EVAL_RECIPES:
    recipe_titles = recipes.filter(['title']).values.ravel().tolist()
    # recipe_words = set()

    # Create vocabulary of words used in recipe titles as potential inputs for generation
    # for title in recipe_titles:
    #     for word in title.split(' '):
    #         if len(word) > 0: # Don't add empty string
    #             recipe_words.add(word)

    # len(recipe_words)

    model_path = get_model_path('gpt2', 'fp16_recipe_20000_lr_5e-5_wd_1e-1')

    eval_output_dir = os.path.join(CACHE_DIR, "gpt2_title_prompt_output_recipes")
    pathlib.Path(eval_output_dir).mkdir(exist_ok=True)

    # Generate N recipes using random word in title vocab as input to model
    # Recipes are saved to tmp/gpt2_finetuned_output_recipes
    N = 500
    used_titles = []

    for i in tqdm(range(N)):
        print(f"> Recipe {i} of {N}")
        keyword = random.choice(tuple(recipe_titles))
        output_file_name = os.path.join(eval_output_dir, f"gpt2_recipe_{i}.txt")
        generate_file('gpt2', model_path, keyword, output_path=output_file_name, temperature=0.8, length=400, prefix='<BOS>', stop_token='<EOS>')

    # Save used titles for later reference
    title_list_file = os.path.join(eval_output_dir, f"titles.txt")
    with open(title_list_file, 'w') as f:
        f.writelines("%s\n" % t for t in used_titles)

  0%|          | 0/500 [00:00<?, ?it/s]

gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1
> Recipe 0 of 500
["--temperature='0.8'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2020-12-18 17:44:34.397758: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
12/18/2020 17:44:36 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
12/18/2020 17:44:46 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/CS 269: Recipe/tmp/gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Mango Lassi II', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.8, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  0%|          | 1/500 [00:19<2:39:41, 19.20s/it]

> Recipe 1 of 500
["--temperature='0.8'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2020-12-18 17:44:53.590156: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
12/18/2020 17:44:55 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
12/18/2020 17:45:05 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/CS 269: Recipe/tmp/gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Too Much Strawberry Cake', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.8, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  0%|          | 2/500 [00:38<2:40:09, 19.30s/it]

...

> Recipe 498 of 500
["--temperature='0.8'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2020-12-18 20:28:20.565802: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
12/18/2020 20:28:22 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
12/18/2020 20:28:32 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/CS 269: Recipe/tmp/gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Honey-Balsamic Chicken', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.8, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


100%|█████████▉| 499/500 [2:44:05<00:19, 19.67s/it]

> Recipe 499 of 500
["--temperature='0.8'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2020-12-18 20:28:39.962230: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
12/18/2020 20:28:42 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
12/18/2020 20:28:52 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/CS 269: Recipe/tmp/gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Cream Of Carrot Soup', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.8, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


100%|██████████| 500/500 [2:44:25<00:00, 19.73s/it]


NameError: ignored